# Heady Systems - Project Quickstart

```
╔═══════════════════════════════════════════════╗
║  ██╗  ██╗███████╗ █████╗ ██████╗ ██╗   ██╗  ║
║  ██║  ██║██╔════╝██╔══██╗██╔══██╗╚██╗ ██╔╝  ║
║  ███████║█████╗  ███████║██║  ██║ ╚████╔╝   ║
║  ██╔══██║██╔══╝  ██╔══██║██║  ██║  ╚██╔╝    ║
║  ██║  ██║███████╗██║  ██║██████╔╝   ██║    ║
║  ╚═╝  ╚═╝╚══════╝╚═╝  ╚═╝╚═════╝    ╚═╝    ║
║                                               ║
║  ∞ SACRED GEOMETRY ∞                            ║
║  Organic Systems · Breathing Interfaces         ║
╚═══════════════════════════════════════════════╝
```

**Version:** 2.0.0 | **Runtime:** GPU (T4 recommended) | **Repo:** [HeadySystems/Heady](https://github.com/HeadySystems/Heady)

---

## What This Notebook Does

| Step | Description |
|------|-------------|
| **1. Environment** | Verify GPU, install system deps |
| **2. Clone & Install** | Pull repo, install Node.js + Python packages |
| **3. Configuration** | Set API keys and environment variables |
| **4. Boot Manager** | Start `heady-manager.js` on port 3300 |
| **5. API Explorer** | Hit health, registry, conductor, and pipeline endpoints |
| **6. HuggingFace Inference** | Run PYTHIA node with GPU-accelerated models |
| **7. Tunnel Access** | Expose the Sacred Geometry UI via ngrok |
| **8. Quickstart Quiz** | Flashcard review of core architecture |

---
## 1. Environment Verification
Confirm GPU availability and print system info.

In [ ]:
import torch
import subprocess, sys, os

print("=" * 60)
print("HEADY SYSTEMS - Environment Check")
print("=" * 60)

# GPU check
gpu_available = torch.cuda.is_available()
gpu_name = torch.cuda.get_device_name(0) if gpu_available else "None"
gpu_mem = f"{torch.cuda.get_device_properties(0).total_mem / 1e9:.1f} GB" if gpu_available else "N/A"

print(f"GPU Available : {gpu_available}")
print(f"GPU Device    : {gpu_name}")
print(f"GPU Memory    : {gpu_mem}")
print(f"PyTorch       : {torch.__version__}")
print(f"CUDA Version  : {torch.version.cuda}")
print(f"Python        : {sys.version.split()[0]}")
print("=" * 60)

if not gpu_available:
    print("\n[WARNING] No GPU detected.")
    print("Go to Runtime > Change runtime type > GPU")
else:
    print(f"\n[OK] GPU ready: {gpu_name} ({gpu_mem})")

---
## 2. Clone Repository & Install Dependencies
Pull the Heady repo and install both Node.js and Python packages.

In [ ]:
%%bash
echo "[1/4] Installing Node.js 20.x..."
curl -fsSL https://deb.nodesource.com/setup_20.x | sudo -E bash - > /dev/null 2>&1
sudo apt-get install -y nodejs > /dev/null 2>&1
echo "  Node: $(node --version)  npm: $(npm --version)"

echo "[2/4] Cloning Heady repository..."
if [ -d "/content/Heady" ]; then
  echo "  Repo already exists, pulling latest..."
  cd /content/Heady && git pull --ff-only
else
  git clone https://github.com/HeadySystems/Heady.git /content/Heady
fi

echo "[3/4] Installing Node.js dependencies..."
cd /content/Heady && npm install --production 2>&1 | tail -1

echo "[4/4] Installing Python dependencies..."
pip install -q -r /content/Heady/backend/python_worker/requirements.txt 2>&1 | tail -3

echo ""
echo "[OK] All dependencies installed."

---
## 3. Configuration
Set environment variables. Replace placeholder values with your actual keys.

In [ ]:
import os
from getpass import getpass

# --- Required ---
# You can paste keys directly or use getpass for security
os.environ["HEADY_API_KEY"] = getpass("Enter HEADY_API_KEY (or press Enter for dev default): ") or "dev_colab_key"
os.environ["ADMIN_TOKEN"] = getpass("Enter ADMIN_TOKEN (or press Enter for dev default): ") or "colab_secure_token"

# --- Optional: Hugging Face ---
hf_token = getpass("Enter HF_TOKEN (or press Enter to skip): ")
if hf_token:
    os.environ["HF_TOKEN"] = hf_token

# --- System config ---
os.environ["PORT"] = "3300"
os.environ["NODE_ENV"] = "development"
os.environ["HEADY_TARGET"] = "colab"
os.environ["HEADY_VERSION"] = "2.0.0-colab"
os.environ["ENABLE_CODEMAP"] = "true"
os.environ["JULES_ENABLED"] = "true"
os.environ["OBSERVER_ENABLED"] = "true"
os.environ["BUILDER_ENABLED"] = "true"
os.environ["ATLAS_ENABLED"] = "true"

# GPU config (auto-detect)
import torch
if torch.cuda.is_available():
    os.environ["HEADY_ADMIN_ENABLE_GPU"] = "true"
    gpu_mem_mb = int(torch.cuda.get_device_properties(0).total_mem / 1e6)
    os.environ["GPU_MEMORY_LIMIT"] = str(gpu_mem_mb)
    print(f"[OK] GPU config set: {gpu_mem_mb} MB")
else:
    os.environ["HEADY_ADMIN_ENABLE_GPU"] = "false"
    print("[INFO] Running in CPU-only mode")

print(f"[OK] Environment configured for Heady {os.environ['HEADY_VERSION']}")

---
## 4. Boot Heady Manager
Start the Node.js MCP server in the background on port 3300.

In [ ]:
import subprocess, time, os, json
from urllib.request import urlopen, Request
from urllib.error import URLError

HEADY_DIR = "/content/Heady"
PORT = int(os.environ.get("PORT", 3300))

# Build env dict for the subprocess
env = {**os.environ, "PORT": str(PORT)}

# Start heady-manager.js in background
print(f"[BOOT] Starting heady-manager.js on port {PORT}...")
proc = subprocess.Popen(
    ["node", "heady-manager.js"],
    cwd=HEADY_DIR,
    env=env,
    stdout=subprocess.PIPE,
    stderr=subprocess.STDOUT
)

# Wait for server to be ready
health_url = f"http://localhost:{PORT}/api/health"
for attempt in range(15):
    time.sleep(2)
    try:
        resp = urlopen(health_url, timeout=3)
        data = json.loads(resp.read())
        if data.get("ok"):
            print(f"[OK] Heady Manager is LIVE (PID: {proc.pid})")
            print(f"     Service : {data.get('service')}")
            print(f"     Version : {data.get('version')}")
            print(f"     Health  : http://localhost:{PORT}/api/health")
            break
    except (URLError, Exception):
        if attempt < 14:
            print(f"     Waiting... ({attempt+1}/15)")
        else:
            print("[ERROR] Server failed to start. Check logs:")
            print(proc.stdout.read(4096).decode())

---
## 5. API Explorer
Interact with the Heady Manager REST API. Each cell hits a different endpoint group.

In [ ]:
import json
from urllib.request import urlopen, Request

BASE = f"http://localhost:{os.environ.get('PORT', 3300)}"

def heady_get(path):
    """GET a Heady API endpoint and pretty-print the JSON response."""
    url = f"{BASE}{path}"
    try:
        resp = urlopen(url, timeout=10)
        data = json.loads(resp.read())
        print(f"GET {path}")
        print(json.dumps(data, indent=2))
        return data
    except Exception as e:
        print(f"[ERROR] {path}: {e}")
        return None

def heady_post(path, body=None):
    """POST to a Heady API endpoint and pretty-print the JSON response."""
    url = f"{BASE}{path}"
    try:
        payload = json.dumps(body or {}).encode()
        req = Request(url, data=payload, headers={"Content-Type": "application/json"})
        resp = urlopen(req, timeout=30)
        data = json.loads(resp.read())
        print(f"POST {path}")
        print(json.dumps(data, indent=2))
        return data
    except Exception as e:
        print(f"[ERROR] {path}: {e}")
        return None

print("Helper functions loaded: heady_get(path), heady_post(path, body)")

### 5a. Health & Pulse

In [ ]:
# Health check
heady_get("/api/health")
print("\n" + "="*60 + "\n")
# System pulse
heady_get("/api/pulse")

### 5b. Registry & Nodes

In [ ]:
# Component registry
heady_get("/api/registry")
print("\n" + "="*60 + "\n")
# Node status
heady_get("/api/nodes")

### 5c. System Status & Production Activation

In [ ]:
# Full system status
status = heady_get("/api/system/status")

if status:
    caps = status.get("capabilities", {})
    print("\n--- Capability Summary ---")
    for key, val in caps.items():
        total = val.get("total", 0)
        active = val.get("active", val.get("healthy", 0))
        print(f"  {key:12s}: {active}/{total} active")

In [ ]:
# Activate production mode (enables all nodes, tools, workflows)
# Uncomment the next line to activate:
# heady_post("/api/system/production")

### 5d. Pipeline & Subsystems

In [ ]:
# Pipeline config
heady_get("/api/pipeline/config")
print("\n" + "="*60 + "\n")
# Pipeline state
heady_get("/api/pipeline/state")
print("\n" + "="*60 + "\n")
# Brain status
heady_get("/api/brain/status")
print("\n" + "="*60 + "\n")
# Readiness evaluation
heady_get("/api/readiness/evaluate")

### 5e. Layer Management

In [ ]:
# Current layer
heady_get("/api/layer")

# Available layers:
#   local      - Local Dev (localhost:3300)
#   cloud-me   - Cloud HeadyMe
#   cloud-sys  - Cloud HeadySystems
#   cloud-conn - Cloud HeadyConnection
#   hybrid     - Hybrid Local+Cloud
#
# Switch layer (uncomment to use):
# heady_post("/api/layer/switch", {"layer": "cloud-sys"})

---
## 6. HeadyConductor - Python Orchestration
Run the HeadyConductor directly to orchestrate AI nodes.

In [ ]:
import subprocess, json

HEADY_DIR = "/content/Heady"
CONDUCTOR = f"{HEADY_DIR}/HeadyAcademy/HeadyConductor.py"

def run_conductor(args: list):
    """Run HeadyConductor with given arguments and return parsed output."""
    result = subprocess.run(
        ["python", CONDUCTOR] + args,
        capture_output=True, text=True, cwd=HEADY_DIR,
        env={**os.environ, "PYTHONUNBUFFERED": "1"}
    )
    print(f"$ python HeadyConductor.py {' '.join(args)}")
    if result.returncode != 0:
        print(f"[ERROR] Exit code {result.returncode}")
        print(result.stderr[-500:] if result.stderr else "No stderr")
        return None
    # Try to extract JSON
    import re
    match = re.search(r'\{[\s\S]*\}', result.stdout)
    if match:
        data = json.loads(match.group(0))
        print(json.dumps(data, indent=2))
        return data
    else:
        print(result.stdout[-1000:])
        return result.stdout

In [ ]:
# System summary
run_conductor(["--summary"])

In [ ]:
# Health check via Conductor
run_conductor(["--health"])

In [ ]:
# Orchestrate a request (routes to the appropriate AI node)
# Example: Ask PYTHIA for inference
run_conductor(["--request", "analyze the system architecture"])

---
## 7. HuggingFace Inference (GPU-Accelerated)
Run the PYTHIA node's HuggingFace tool directly with GPU acceleration.

In [ ]:
import torch
from transformers import pipeline

device = 0 if torch.cuda.is_available() else -1
print(f"Using device: {'GPU (cuda:0)' if device == 0 else 'CPU'}")

# Text generation with a small model
generator = pipeline("text-generation", model="distilgpt2", device=device)

prompt = "Heady Systems uses Sacred Geometry architecture to"
result = generator(prompt, max_length=80, num_return_sequences=1, do_sample=True, temperature=0.7)

print("\n--- Generated Text ---")
print(result[0]["generated_text"])

In [ ]:
# Sentiment analysis
classifier = pipeline("sentiment-analysis", device=device)

texts = [
    "The Sacred Geometry architecture makes everything interconnected and beautiful.",
    "The system health check failed with critical errors.",
    "All nodes are active and the pipeline completed successfully."
]

print("--- Sentiment Analysis ---")
for text, result in zip(texts, classifier(texts)):
    print(f"  [{result['label']:8s} {result['score']:.3f}] {text[:60]}...")

In [ ]:
# Summarization (uses T5-small, good for Colab GPU)
summarizer = pipeline("summarization", model="t5-small", device=device)

architecture_text = """
Heady Systems is a hybrid Node.js and Python system for the HeadyConnection
ecosystem with a patented Sacred Geometry architecture. It features a web-based
Admin IDE with AI assistance, real-time build and audit monitoring, and optional
remote GPU support. The system is orchestrated by heady-manager.js, a Node.js
Express MCP server running on port 3300. It manages AI nodes including JULES for
code optimization, OBSERVER for monitoring, BUILDER for project construction,
ATLAS for documentation, and PYTHIA for predictive inference via HuggingFace.
The HeadyConductor routes requests to appropriate nodes based on trigger keywords.
The system deploys to Render.com using infrastructure-as-code via render.yaml
and supports multiple cloud layers including HeadyMe, HeadySystems, and
HeadyConnection endpoints.
"""

summary = summarizer(architecture_text, max_length=60, min_length=20)
print("--- Architecture Summary ---")
print(summary[0]["summary_text"])

---
## 8. Tunnel Access (ngrok)
Expose the Heady UI publicly so you can access the Sacred Geometry dashboard from your browser.

In [ ]:
# Install pyngrok
!pip install -q pyngrok

from getpass import getpass
ngrok_token = getpass("Enter your ngrok authtoken (from https://dashboard.ngrok.com): ")

if ngrok_token:
    from pyngrok import ngrok, conf
    conf.get_default().auth_token = ngrok_token

    PORT = int(os.environ.get("PORT", 3300))
    tunnel = ngrok.connect(PORT)
    public_url = tunnel.public_url

    print("\n" + "=" * 60)
    print("HEADY SYSTEMS - Public Access")
    print("=" * 60)
    print(f"  Main UI  : {public_url}")
    print(f"  Admin IDE: {public_url}/admin")
    print(f"  Health   : {public_url}/api/health")
    print(f"  Pulse    : {public_url}/api/pulse")
    print("=" * 60)
    print("\nKeep this notebook running to maintain the tunnel.")
else:
    print("[SKIP] No ngrok token provided. Access via Colab proxy only.")
    print(f"  Local URL: http://localhost:{os.environ.get('PORT', 3300)}")

---
## 9. Run a Pipeline
Trigger the HCFullPipeline which runs all stages: Supervisor routing, Brain auto-tuning, Health checks, Readiness evaluation, and Checkpoint analysis.

In [ ]:
import time

# Trigger pipeline run
run_result = heady_post("/api/pipeline/run")

if run_result and run_result.get("accepted"):
    run_id = run_result["runId"]
    print(f"\nPipeline started: {run_id}")
    print("Polling for completion...\n")

    for i in range(30):
        time.sleep(2)
        state = heady_get("/api/pipeline/state") if i % 5 == 0 else None
        if state is None:
            # Silent poll
            import json
            from urllib.request import urlopen
            try:
                resp = urlopen(f"{BASE}/api/pipeline/state", timeout=5)
                state = json.loads(resp.read())
            except:
                continue

        status = state.get("status", "unknown") if state else "unknown"
        if status in ["completed", "failed", "idle"]:
            print(f"\n[DONE] Pipeline status: {status}")
            if state and state.get("metrics"):
                print(f"  Metrics: {json.dumps(state['metrics'], indent=2)}")
            break
    else:
        print("[TIMEOUT] Pipeline still running after 60s. Check /api/pipeline/state")

---
## 10. Quickstart Quiz - Flashcard Review

Per the **Heady Documentation Protocol** (Quiz & Flashcard Methodology), here are the core architecture flashcards.

---

### Architecture

| # | Question | Answer |
|---|----------|--------|
| 1 | What is the primary orchestrator of the Heady system? | `heady-manager.js` - a Node.js Express MCP server on port 3300 |
| 2 | What design philosophy does the Heady UI follow? | **Sacred Geometry** - rounded, organic, breathing interfaces |
| 3 | What two runtimes does Heady use? | **Node.js** (manager/API) and **Python** (worker/inference) |
| 4 | Where is the infrastructure-as-code deployment defined? | `render.yaml` (Render.com Blueprint) |
| 5 | What is the health check endpoint? | `GET /api/health` returns `{"ok": true, "service": "heady-manager"}` |

### AI Nodes

| # | Question | Answer |
|---|----------|--------|
| 6 | Name the 5 AI nodes in the system. | **JULES** (Surgeon), **OBSERVER** (Monitor), **BUILDER** (Constructor), **ATLAS** (Archivist), **PYTHIA** (Oracle) |
| 7 | Which node handles HuggingFace inference? | **PYTHIA** - The Oracle, triggered by `huggingface`, `predict`, `ask_oracle` |
| 8 | What does JULES do? | Code optimization: unused import detection, quality analysis, performance suggestions, security scanning |
| 9 | How does HeadyConductor route requests? | Matches trigger keywords in the request to the appropriate AI node's trigger list |
| 10 | What tool does ATLAS use? | `auto_doc` - generates API docs, performs code analysis, creates knowledge base entries |

### Pipeline & Subsystems

| # | Question | Answer |
|---|----------|--------|
| 11 | What is HCFullPipeline? | The main execution engine with stage DAG, circuit breakers, checkpoints, and event-driven subsystem feedback |
| 12 | Name the 5 pipeline subsystems. | **Supervisor**, **Brain**, **CheckpointAnalyzer**, **ReadinessEvaluator**, **HealthRunner** |
| 13 | What does the Brain subsystem do? | Auto-tunes system parameters based on error rate, latency, and queue utilization |
| 14 | How do you trigger a pipeline run via API? | `POST /api/pipeline/run` - returns immediately with `runId`, runs asynchronously |
| 15 | What is a checkpoint in the pipeline? | An analysis point where CheckpointAnalyzer evaluates run state and health, feeding results into Brain |

### Cloud Layers

| # | Question | Answer |
|---|----------|--------|
| 16 | What are the 5 cloud layers? | `local`, `cloud-me`, `cloud-sys`, `cloud-conn`, `hybrid` |
| 17 | How do you switch layers via API? | `POST /api/layer/switch` with body `{"layer": "cloud-sys"}` |
| 18 | Where is the active layer state persisted? | `scripts/.heady-active-layer` file |
| 19 | What does production activation do? | Activates ALL nodes, tools, workflows, services and sets environment to production |
| 20 | What endpoint returns the full system status? | `GET /api/system/status` - includes capabilities, layer info, Sacred Geometry state |

In [ ]:
# Interactive quiz mode
import random

flashcards = [
    ("What is the primary orchestrator of the Heady system?",
     "heady-manager.js - a Node.js Express MCP server on port 3300"),
    ("What design philosophy does the Heady UI follow?",
     "Sacred Geometry - rounded, organic, breathing interfaces"),
    ("What two runtimes does Heady use?",
     "Node.js (manager/API) and Python (worker/inference)"),
    ("Name the 5 AI nodes.",
     "JULES (Surgeon), OBSERVER (Monitor), BUILDER (Constructor), ATLAS (Archivist), PYTHIA (Oracle)"),
    ("Which node handles HuggingFace inference?",
     "PYTHIA - The Oracle, triggered by: huggingface, predict, ask_oracle"),
    ("What is HCFullPipeline?",
     "Main execution engine with stage DAG, circuit breakers, checkpoints, and event-driven feedback"),
    ("Name the 5 pipeline subsystems.",
     "Supervisor, Brain, CheckpointAnalyzer, ReadinessEvaluator, HealthRunner"),
    ("What are the 5 cloud layers?",
     "local, cloud-me, cloud-sys, cloud-conn, hybrid"),
    ("How do you activate production mode via API?",
     "POST /api/system/production - activates all nodes, tools, workflows, services"),
    ("Where is the infrastructure-as-code deployment defined?",
     "render.yaml (Render.com Blueprint)"),
]

print("=" * 60)
print("HEADY SYSTEMS - Interactive Quiz (10 Questions)")
print("=" * 60)
print("Press Enter after each question to reveal the answer.\n")

random.shuffle(flashcards)
score = 0

for i, (q, a) in enumerate(flashcards, 1):
    print(f"Q{i}: {q}")
    input("  [Press Enter to reveal answer]")
    print(f"  A: {a}")
    got_it = input("  Did you know it? (y/n): ").strip().lower()
    if got_it == "y":
        score += 1
    print()

print("=" * 60)
pct = (score / len(flashcards)) * 100
print(f"Score: {score}/{len(flashcards)} ({pct:.0f}%)")
if pct >= 80:
    print("Sacred Geometry Mastery Achieved!")
elif pct >= 50:
    print("Good progress. Review the Architecture section above.")
else:
    print("Re-run the notebook and study the flashcard table in Section 10.")
print("=" * 60)

---
## Cleanup
Stop the server and tunnel when done.

In [ ]:
# Stop ngrok tunnel
try:
    from pyngrok import ngrok
    ngrok.kill()
    print("[OK] ngrok tunnel closed.")
except:
    pass

# Stop heady-manager
try:
    proc.terminate()
    proc.wait(timeout=5)
    print(f"[OK] heady-manager stopped (PID: {proc.pid}).")
except:
    print("[INFO] Server may already be stopped.")

print("\nSession complete. Sacred Geometry resting.")